# Laboratorio 8 - Transformaciones con Spark

#### Edwin Ortega 22305
#### Esteban Zambrano 22119
#### Diego García 22404

%md
### Carga de Datos y Análisis Exploratorio

%md
##### Instalación a tener en cuenta

In [0]:
#%pip install openpyxl

%md
### Convertir Excel a un CSV por hoja
##### Rutas, imports y utilidades 

Se debe mover los excel en la carpeta 'Data' a un volumen de databricks. Se deben cambiar las rutas a las suyas.

In [0]:
import os
import pandas as pd
from pathlib import Path

# Cambiar esto a su volumen
BASE_DIR = "/Volumes/workspace/default/lab8/"

SUBFOLDERS = ["fallecidos-lesionados", "hechos", "vehiculos"]
OUTPUT_ROOT = os.path.join(BASE_DIR, "csv")

In [0]:
os.makedirs(OUTPUT_ROOT, exist_ok=True)
for sub in SUBFOLDERS:
    os.makedirs(os.path.join(OUTPUT_ROOT, sub), exist_ok=True)

print("Base:", BASE_DIR)
print("Salida:", OUTPUT_ROOT)
print("Subcarpetas:", SUBFOLDERS)


In [0]:
manifest, errors = [], []

for sub in SUBFOLDERS:
    in_dir = Path(BASE_DIR) / sub
    out_dir = Path(OUTPUT_ROOT) / sub

    for f in sorted(in_dir.iterdir()):
        if not f.is_file():
            continue
        if f.suffix.lower() not in {".xlsx", ".xls"}:
            continue

        try:
            # Lee SOLO la primera hoja
            df = pd.read_excel(f, sheet_name=0)
            # Nombre base del archivo + .csv
            csv_name = f.with_suffix(".csv").name
            csv_path = out_dir / csv_name

            # Guarda CSV sin índice
            df.to_csv(csv_path, index=False)
            manifest.append((sub, str(f), str(csv_path)))
        except Exception as e:
            errors.append((str(f), repr(e)))

print(f"CSV generados: {len(manifest)}")
if errors:
    print(f"Archivos con error: {len(errors)} (muestra 5)")
    for p, err in errors[:5]:
        print("-", p, "->", err)

##### Limpieza de datos

In [0]:
INPUT_ROOT  = Path(BASE_DIR) / "csv"

# Recolecta rutas de todos los CSV
csv_files = []
for sub in SUBFOLDERS:
    folder = INPUT_ROOT / sub
    if not folder.exists():
        print(f"⚠️ No existe: {folder}")
        continue
    for f in sorted(folder.iterdir()):
        if f.is_file() and f.suffix.lower() == ".csv":
            csv_files.append((sub, f))

print(f"Archivos CSV encontrados: {len(csv_files)}")
for i, (sub, f) in enumerate(csv_files[:10], start=1):
    print(f"{i:02d}. [{sub}] {f.name}")


In [0]:
# Estandarización de datos

import unicodedata
import re

def strip_accents_lower(text: str) -> str:
    """Convierte a minúsculas, elimina acentos/diéresis y comprime espacios."""
    if text is None:
        return None
    s = str(text).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = s.encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"\s+", " ", s)
    return s

def normalize_headers(cols):
    """Normaliza encabezados: minus, sin acentos, espacios->_, solo [a-z0-9_]."""
    norm = []
    for c in cols:
        s = strip_accents_lower(c)
        s = re.sub(r"[^a-z0-9_ ]", "", s)
        s = s.replace(" ", "_")
        s = re.sub(r"_+", "_", s).strip("_")
        norm.append(s or "col")
    return norm

def normalize_dataframe_text(df: pd.DataFrame) -> pd.DataFrame:
    """Devuelve una copia con encabezados normalizados y columnas object en minus/sin acentos."""
    out = df.copy()
    out.columns = normalize_headers(out.columns)
    obj_cols = out.select_dtypes(include=["object"]).columns.tolist()
    for c in obj_cols:
        out[c] = out[c].map(strip_accents_lower)
    return out


In [0]:
raw_data   = {}
clean_data = {}

errors = []

for sub, f in csv_files:
    try:
        # Detecta separador automáticamente; cae a coma si falla
        try:
            df = pd.read_csv(f, sep=None, engine="python", encoding="utf-8", encoding_errors="ignore")
        except Exception:
            df = pd.read_csv(f, encoding="utf-8", encoding_errors="ignore")

        df_clean = normalize_dataframe_text(df)

        key = (sub, f.name)
        raw_data[key] = df
        clean_data[key] = df_clean

    except Exception as e:
        errors.append((str(f), repr(e)))

print(f"DataFrames cargados: {len(raw_data)} | Normalizados: {len(clean_data)}")
if errors:
    print(f"⚠️ Errores en {len(errors)} archivos (muestra 5):")
    for p, err in errors[:5]:
        print("-", p, "->", err)


In [0]:
# Selección de columanas para Fallecidos/lesionados

import re
import pandas as pd

# columnas requeridas
req_fall = ["ano_ocu","mes_ocu","depto_ocu","zona_ocu","edad_per","tipo_eve","fall_les"]

sel_fallecidos = {}
faltantes_fallecidos = []

for (sub, fname), df in clean_data.items():
    if sub != "fallecidos-lesionados":
        continue

    # detecta año desde el nombre del archivo (primer 4 dígitos)
    m = re.search(r"(\d{4})", fname)
    anio = int(m.group(1)) if m else None

    # crea columnas faltantes con "ignorado"
    missing = [c for c in req_fall if c not in df.columns]
    for c in missing:
        df[c] = "ignorado"

    # registra faltantes (uno por columna faltante)
    for c in missing:
        faltantes_fallecidos.append({"anio": anio, "archivo": fname, "col_faltante": c})

    # deja solo las columnas requeridas en el orden pedido
    sel = df[req_fall].copy()
    sel_fallecidos[(sub, fname)] = sel

# reporte
reporte_fallecidos = pd.DataFrame(faltantes_fallecidos)
display(reporte_fallecidos.sort_values(["anio","col_faltante"]) if not reporte_fallecidos.empty 
        else pd.DataFrame(columns=["anio","archivo","col_faltante"]))


In [0]:
# Selección de columanas para hechos

import re
import pandas as pd

req_hechos = ["ano_ocu","hora_ocu","mes_ocu","dia_sem_ocu","depto_ocu","tipo_eve"]

sel_hechos = {}
faltantes_hechos = []

for (sub, fname), df in clean_data.items():
    if sub != "hechos":
        continue

    m = re.search(r"(\d{4})", fname)
    anio = int(m.group(1)) if m else None

    missing = [c for c in req_hechos if c not in df.columns]
    for c in missing:
        df[c] = "ignorado"

    for c in missing:
        faltantes_hechos.append({"anio": anio, "archivo": fname, "col_faltante": c})

    sel = df[req_hechos].copy()
    sel_hechos[(sub, fname)] = sel

reporte_hechos = pd.DataFrame(faltantes_hechos)
display(reporte_hechos.sort_values(["anio","col_faltante"]) if not reporte_hechos.empty 
        else pd.DataFrame(columns=["anio","archivo","col_faltante"]))


In [0]:
# Selección de columanas para vehiculos

import re
import pandas as pd

req_veh = ["ano_ocu","mes_ocu","depto_ocu","sexo_per","tipo_veh","marca_veh","color_veh","modelo_veh","tipo_eve"]

sel_vehiculos = {}
faltantes_vehiculos = []

for (sub, fname), df in clean_data.items():
    if sub != "vehiculos":
        continue

    m = re.search(r"(\d{4})", fname)
    anio = int(m.group(1)) if m else None

    missing = [c for c in req_veh if c not in df.columns]
    for c in missing:
        df[c] = "ignorado"

    for c in missing:
        faltantes_vehiculos.append({"anio": anio, "archivo": fname, "col_faltante": c})

    sel = df[req_veh].copy()
    sel_vehiculos[(sub, fname)] = sel

reporte_vehiculos = pd.DataFrame(faltantes_vehiculos)
display(reporte_vehiculos.sort_values(["anio","col_faltante"]) if not reporte_vehiculos.empty 
        else pd.DataFrame(columns=["anio","archivo","col_faltante"]))


In [0]:
import re

YEAR = 2019  # ← cambia el año

keys_veh = [k for k in sel_vehiculos.keys() if re.search(fr"\b{YEAR}\b", k[1])]
keys_veh = sorted(keys_veh, key=lambda x: x[1])

if keys_veh:
    sample_key = keys_veh[0]
    print("Mostrando (vehiculos):", sample_key)
    display(sel_vehiculos[sample_key].head(10))
else:
    print(f"No se encontró archivo de vehiculos con año {YEAR} en el nombre.")


In [0]:
# Estandarización de datos generales

import re
import pandas as pd

YEAR_MIN, YEAR_MAX = 2013, 2020

# mapas
MES_MAP = {
    1:"enero", 2:"febrero", 3:"marzo", 4:"abril", 5:"mayo", 6:"junio",
    7:"julio", 8:"agosto", 9:"septiembre", 10:"octubre", 11:"noviembre", 12:"diciembre"
}

DEPTO_MAP = {
     1:"guatemala",  2:"el progreso", 3:"sacatepequez", 4:"chimaltenango",
     5:"escuintla",  6:"santa rosa",  7:"solola",       8:"totonicapan",
     9:"quetzaltenango", 10:"suchitepequez", 11:"retalhuleu", 12:"san marcos",
    13:"huehuetenango", 14:"quiche", 15:"baja verapaz", 16:"alta verapaz",
    17:"peten", 18:"izabal", 19:"zacapa", 20:"chiquimula", 21:"jalapa", 22:"jutiapa"
}

TIPO_EVE_MAP = {
     1:"colision", 2:"choque", 3:"vuelco", 4:"caida", 5:"atropello",
     6:"perdida de control", 7:"colision contra animal", 8:"exceso de pasaje",
     9:"asfalto mojado", 10:"exceso de velocidad", 11:"desperfectos mecanicos",
    12:"incendio", 99:"ignorado"
}

def _apply_general_changes(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 1) "ignorada" -> "ignorado" en todas las columnas de texto
    obj_cols = out.select_dtypes(include=["object"]).columns.tolist()
    for c in obj_cols:
        out[c] = out[c].replace("ignorada", "ignorado")

    # 2) mes_ocu numérico -> nombre
    if "mes_ocu" in out.columns:
        _num = pd.to_numeric(out["mes_ocu"], errors="coerce")
        mask = _num.notna()
        out.loc[mask, "mes_ocu"] = _num[mask].astype(int).map(MES_MAP).fillna(out.loc[mask, "mes_ocu"])

    # 3) depto_ocu numérico -> nombre
    if "depto_ocu" in out.columns:
        _num = pd.to_numeric(out["depto_ocu"], errors="coerce")
        mask = _num.notna()
        out.loc[mask, "depto_ocu"] = _num[mask].astype(int).map(DEPTO_MAP).fillna(out.loc[mask, "depto_ocu"])

    # 4) tipo_eve numérico -> nombre
    if "tipo_eve" in out.columns:
        _num = pd.to_numeric(out["tipo_eve"], errors="coerce")
        mask = _num.notna()
        out.loc[mask, "tipo_eve"] = _num[mask].astype(int).map(TIPO_EVE_MAP).fillna(out.loc[mask, "tipo_eve"])

    return out

def _year_from_filename(fname: str):
    m = re.search(r"(\d{4})", fname)
    return int(m.group(1)) if m else None


changed_fallecidos = {}
changed_hechos = {}
changed_vehiculos = {}

for key, df in sel_fallecidos.items():
    sub, fname = key
    yr = _year_from_filename(fname)
    if yr is not None and (YEAR_MIN <= yr <= YEAR_MAX):
        changed_fallecidos[key] = _apply_general_changes(df)
    else:
        changed_fallecidos[key] = df

for key, df in sel_hechos.items():
    sub, fname = key
    yr = _year_from_filename(fname)
    if yr is not None and (YEAR_MIN <= yr <= YEAR_MAX):
        changed_hechos[key] = _apply_general_changes(df)
    else:
        changed_hechos[key] = df

for key, df in sel_vehiculos.items():
    sub, fname = key
    yr = _year_from_filename(fname)
    if yr is not None and (YEAR_MIN <= yr <= YEAR_MAX):
        changed_vehiculos[key] = _apply_general_changes(df)
    else:
        changed_vehiculos[key] = df


In [0]:
# Estandarización de datos de fallecidos-lesionados

import re
import pandas as pd

YEAR_MIN, YEAR_MAX = 2013, 2020

def _year_from_filename(fname: str):
    m = re.search(r"(\d{4})", fname)
    return int(m.group(1)) if m else None

final_fallecidos = {}

for key, df in changed_fallecidos.items():
    sub, fname = key
    yr = _year_from_filename(fname)

    if yr is None or not (YEAR_MIN <= yr <= YEAR_MAX):
        # fuera de rango: lo dejamos igual
        final_fallecidos[key] = df
        continue

    out = df.copy()

    # zona_ocu: 99 -> "ignorado"
    if "zona_ocu" in out.columns:
        znum = pd.to_numeric(out["zona_ocu"], errors="coerce")
        out.loc[znum == 99, "zona_ocu"] = "ignorado"

    # edad_per: 999 -> "ignorado"
    if "edad_per" in out.columns:
        ednum = pd.to_numeric(out["edad_per"], errors="coerce")
        out.loc[ednum == 999, "edad_per"] = "ignorado"

    # fall_les: 1 -> "fallecido"; 2 -> "lesionado"
    if "fall_les" in out.columns:
        # Maneja valores numéricos o string
        out["fall_les"] = (
            out["fall_les"]
            .astype(str).str.strip()
            .replace({"1": "fallecido", "2": "lesionado"})
        )

    for c in ["zona_ocu", "edad_per", "fall_les", "mes_ocu","depto_ocu", "tipo_eve"]:
        if c in out.columns:
            out[c] = out[c].astype("string").fillna("ignorado")

    final_fallecidos[key] = out

# Vista rápida
if final_fallecidos:
    sample_key = sorted(final_fallecidos.keys(), key=lambda k: k[1])[0]
    print("Mostrando (fallecidos-lesionados):", sample_key)
    display(final_fallecidos[sample_key].head(10))


In [0]:
# Estandarización de datos de hechos

import re
import pandas as pd

YEAR_MIN, YEAR_MAX = 2013, 2020

def _year_from_filename(fname: str):
    m = re.search(r"(\d{4})", fname)
    return int(m.group(1)) if m else None

# 1=lunes, 2=martes, ..., 7=domingo (sin tildes)
DOW_MAP = {1:"lunes", 2:"martes", 3:"miercoles", 4:"jueves", 5:"viernes", 6:"sabado", 7:"domingo"}

final_hechos = {}

for key, df in changed_hechos.items():
    sub, fname = key
    yr = _year_from_filename(fname)

    if yr is None or not (YEAR_MIN <= yr <= YEAR_MAX):
        final_hechos[key] = df
        continue

    out = df.copy()

    if "dia_sem_ocu" in out.columns:
        nums = pd.to_numeric(out["dia_sem_ocu"], errors="coerce")
        mask = nums.notna()
        out.loc[mask, "dia_sem_ocu"] = nums[mask].astype(int).map(DOW_MAP).fillna(out.loc[mask, "dia_sem_ocu"])
        # Evita problemas Arrow: homogeniza a string y rellena nulos
        out["dia_sem_ocu"] = out["dia_sem_ocu"].astype("string").fillna("ignorado")

    final_hechos[key] = out

# Vista rápida
if final_hechos:
    sample_key = sorted(final_hechos.keys(), key=lambda k: k[1])[1]
    print("Mostrando (hechos):", sample_key)
    display(final_hechos[sample_key].head(10))


In [0]:
# Estandarización de datos de vehiculos

import re
import pandas as pd

# Rangos
YEAR_MIN_GENERAL, YEAR_MAX_GENERAL = 2013, 2020
YEAR_MIN_DROP_MARCA, YEAR_MAX_DROP_MARCA = 2013, 2023

def _year_from_filename(fname: str):
    m = re.search(r"(\d{4})", fname)
    return int(m.group(1)) if m else None

# Mapas (texto ya sin acentos y en minusculas)
SEXO_MAP = {1: "hombre", 2: "mujer", 9: "ignorado"}

TIPO_VEH_MAP = {
     1: "automovil",  2: "camioneta",  3: "pick_up",      4: "motocicleta",
     5: "camion",     6: "cabezal",    7: "bus_extraurbano", 8: "jeep",
     9: "microbus",  10: "taxi",      11: "panel",       12: "bus_urbano",
    13: "tractor",   14: "moto_taxi", 15: "furgon",      16: "grua",
    17: "bus_escolar", 18: "bicicleta", 99: "ignorado"
}

COLOR_VEH_MAP = {
     1: "rojo",       2: "blanco",     3: "azul",        4: "gris",
     5: "negro",      6: "verde",      7: "amarillo",    8: "celeste",
     9: "corinto",   10: "cafe",      11: "beige",      12: "turquesa",
    13: "marfil",    14: "anaranjado", 15: "aqua",       16: "morado",
    17: "rosado",    99: "ignorado"
}

final_vehiculos = {}

for key, df in changed_vehiculos.items():
    sub, fname = key
    yr = _year_from_filename(fname)
    out = df.copy()

    # ---- Reglas generales para 2013–2020 ----
    if yr is not None and (YEAR_MIN_GENERAL <= yr <= YEAR_MAX_GENERAL):
        # sexo_per
        if "sexo_per" in out.columns:
            _num = pd.to_numeric(out["sexo_per"], errors="coerce")
            mask = _num.notna()
            out.loc[mask, "sexo_per"] = _num[mask].astype(int).map(SEXO_MAP).fillna(out.loc[mask, "sexo_per"])

        # tipo_veh
        if "tipo_veh" in out.columns:
            _num = pd.to_numeric(out["tipo_veh"], errors="coerce")
            mask = _num.notna()
            out.loc[mask, "tipo_veh"] = _num[mask].astype(int).map(TIPO_VEH_MAP).fillna(out.loc[mask, "tipo_veh"])

        # color_veh
        if "color_veh" in out.columns:
            _num = pd.to_numeric(out["color_veh"], errors="coerce")
            mask = _num.notna()
            out.loc[mask, "color_veh"] = _num[mask].astype(int).map(COLOR_VEH_MAP).fillna(out.loc[mask, "color_veh"])

    # ---- Eliminar marca_veh para 2013–2023 ----
    if yr is not None and (YEAR_MIN_DROP_MARCA <= yr <= YEAR_MAX_DROP_MARCA):
        if "marca_veh" in out.columns:
            out = out.drop(columns=["marca_veh"])
        elif "modelo_veh" in out.columns:
            out = out.drop(columns=["modelo_veh"])

    # Homogeneizar tipos a string para evitar errores Arrow
    for c in ["sexo_per", "tipo_veh", "color_veh"]:
        if c in out.columns:
            out[c] = out[c].astype("string").fillna("ignorado")

    final_vehiculos[key] = out

# Vista rápida
if final_vehiculos:
    sample_key = sorted(final_vehiculos.keys(), key=lambda k: k[1])[1]
    print("Mostrando (vehiculos):", sample_key)
    display(final_vehiculos[sample_key].head(10))


In [0]:
# Sobreescribir fallecidos/lesionados

import os
from pathlib import Path

BASE_DIR = "/Volumes/workspace/default/lab8"
OUT_ROOT = Path(BASE_DIR) / "csv"

saved = 0
for (sub, fname), df in final_fallecidos.items():
    out_path = OUT_ROOT / sub / fname
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=False, encoding="utf-8")
    saved += 1

print(f"CSV sobrescritos (fallecidos-lesionados): {saved}")


In [0]:
# Sobreescribir hechos

import os
from pathlib import Path

BASE_DIR = "/Volumes/workspace/default/lab8"
OUT_ROOT = Path(BASE_DIR) / "csv"

saved = 0
for (sub, fname), df in final_hechos.items():
    out_path = OUT_ROOT / sub / fname
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=False, encoding="utf-8")
    saved += 1

print(f"CSV sobrescritos (hechos): {saved}")

In [0]:
# Sobreescribir vehiculos

import os
from pathlib import Path

BASE_DIR = "/Volumes/workspace/default/lab8"
OUT_ROOT = Path(BASE_DIR) / "csv"

saved = 0
for (sub, fname), df in final_vehiculos.items():
    out_path = OUT_ROOT / sub / fname
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=False, encoding="utf-8")
    saved += 1

print(f"CSV sobrescritos (vehiculos): {saved}")


#### Setup

In [0]:
# ================== SETUP CSV → SPARK ==================
# Lee todos los CSV desde tu Unity Catalog Volume sin usar "file:" ni /FileStore.
from pyspark.sql import functions as F

# Lee TODOS los CSV por carpeta (varios años) con wildcard
HECHOS_GLOB    = "/Volumes/workspace/default/lab8/csv/hechos/*.csv"
VEHICULOS_GLOB = "/Volumes/workspace/default/lab8/csv/vehiculos/*.csv"

hechos    = spark.read.csv(HECHOS_GLOB, header=True, inferSchema=True)
vehiculos = spark.read.csv(VEHICULOS_GLOB, header=True, inferSchema=True)

# Normaliza nombres canónicos que usan los incisos
def latin_simplify(s: str) -> str:
    return (s.strip().lower()
            .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
            .replace("ñ","n").replace(" ","_"))

canon_map = {
    "anio": {"anio","ano","year"},
    "mes": {"mes","month"},
    "departamento": {"departamento","depto","dpto"},
    "municipio": {"municipio"},
    "tipo_accidente": {"tipo_accidente","tipo_de_accidente","accidente_tipo","clase_accidente","tipo"},
    "dia_semana": {"dia_semana","dia_de_semana","weekday","day_of_week"},
    "hora": {"hora","hour"},
    "valor": {"valor","cantidad","conteo","n","total"},
    "color_vehiculo": {"color_vehiculo","color","vehiculo_color"},
    "sexo_conductor": {"sexo_conductor","sexo","genero"},
}

def rename_like(df):
    cols = df.columns
    norm = {c: latin_simplify(c) for c in cols}
    inverse = {}
    for canon, options in canon_map.items():
        for c, n in norm.items():
            if n in options and canon not in inverse:
                inverse[canon] = c
    out = df
    for canon, orig in inverse.items():
        if orig and latin_simplify(orig) != canon:
            out = out.withColumnRenamed(orig, canon)
    print("↪︎ Mapeo a canónicas:", {k: inverse.get(k) for k in canon_map})
    return out

hechos    = rename_like(hechos)
vehiculos = rename_like(vehiculos)

# Limpieza mínima: mayúsculas y tipos
for col in ["departamento","municipio","tipo_accidente","dia_semana","color_vehiculo","sexo_conductor"]:
    if col in hechos.columns:    hechos = hechos.withColumn(col, F.upper(F.trim(F.col(col))))
    if col in vehiculos.columns: vehiculos = vehiculos.withColumn(col, F.upper(F.trim(F.col(col))))

if "hora" in hechos.columns: hechos = hechos.withColumn("hora", F.col("hora").cast("int"))
if "anio" in hechos.columns: hechos = hechos.withColumn("anio", F.col("anio").cast("int"))
if "anio" in vehiculos.columns: vehiculos = vehiculos.withColumn("anio", F.col("anio").cast("int"))

print("Listo: 'hechos' y 'vehiculos' cargados desde /Volumes (sin 'file:').")
display(hechos.limit(5))
display(vehiculos.limit(5))

In [0]:
# === Diagnóstico rápido de columnas y renombrado "por contains" ===
from pyspark.sql import functions as F

def show_cols(df, name):
    print(f"\n[{name}] columnas ({len(df.columns)}):")
    print(", ".join(df.columns))

show_cols(hechos, "hechos")
show_cols(vehiculos, "vehiculos")

# Mapeo flexible: busca substrings típicos en nombres de columna (ya sin acentos/espacios)
import unicodedata, re

def norm(s):
    s = "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))
    s = s.lower().strip().replace(" ", "_")
    s = re.sub(r"[^a-z0-9_]", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

# patrones (regex "contains") para cada canónica
PAT = {
    "anio":          r"(anio|a[o|ñ]o|year)\b",
    "mes":           r"\bmes\b|month|mes_num|mesnumero|mes_n|mes_?\d",
    "departamento":  r"depto|dpto|departament",
    "municipio":     r"municip",
    "tipo_accidente":r"tipo.*acciden|clase.*acciden|accidente.*tipo",
    "dia_semana":    r"dia.*semana|weekday|day_?of_?week",
    "hora":          r"^hora$|hour\b|hora_?\d",
    "valor":         r"valor|cantidad|conteo|total|n(_|$)",
    "color_vehiculo":r"color.*vehic|vehic.*color|\bcolor\b",
    "sexo_conductor":r"(sexo|genero).*conduc|conduc.*(sexo|genero)|^sexo$|^genero$",
}

def smart_rename(df):
    cols = df.columns
    nmap = {}   # canónica -> original
    ncols = {c: norm(c) for c in cols}
    for canon, rx in PAT.items():
        rx_comp = re.compile(rx)
        hit = next((c for c in cols if rx_comp.search(ncols[c]) or rx_comp.search(c.lower())), None)
        if hit:
            nmap[canon] = hit

    # Aplica renombres solo si el nombre original difiere
    out = df
    for canon, orig in nmap.items():
        if orig != canon:
            out = out.withColumnRenamed(orig, canon)
    print("Mapeo flexible:", nmap)
    return out

hechos    = smart_rename(hechos)
vehiculos = smart_rename(vehiculos)

# Normalizaciones útiles
for col in ["departamento","municipio","tipo_accidente","dia_semana","color_vehiculo","sexo_conductor"]:
    if col in hechos.columns:    hechos = hechos.withColumn(col, F.upper(F.trim(F.col(col))))
    if col in vehiculos.columns: vehiculos = vehiculos.withColumn(col, F.upper(F.trim(F.col(col))))

# Tipos y derivaciones
if "hora" in hechos.columns: hechos = hechos.withColumn("hora", F.col("hora").cast("int"))
if "anio" in hechos.columns: hechos = hechos.withColumn("anio", F.col("anio").cast("int"))
if "anio" in vehiculos.columns: vehiculos = vehiculos.withColumn("anio", F.col("anio").cast("int"))

# Si no hay 'anio' o 'mes' pero sí hay 'fecha', derivarlos
for df_name, df in [("hechos", hechos), ("vehiculos", vehiculos)]:
    cols = df.columns
    fecha_like = next((c for c in cols if norm(c) in {"fecha","fecha_accidente","fec_hecho","f_accidente"}), None)
    if fecha_like:
        ts = F.to_timestamp(F.col(fecha_like))
        if "anio" not in cols:
            df = df.withColumn("anio", F.year(ts))
        if "mes" not in cols:
            df = df.withColumn("mes", F.month(ts))
        if df_name == "hechos":
            hechos = df
        else:
            vehiculos = df

print("\nEsquemas tras normalización:")
hechos.printSchema()
vehiculos.printSchema()

display(hechos.limit(5))
display(vehiculos.limit(5))


In [0]:
from pyspark.sql import functions as F

# ---------- Normalización mínima ----------
hechos = (hechos
    .withColumn("ano_ocu", F.col("ano_ocu").cast("int"))
    .withColumn("departamento", F.upper(F.trim(F.col("departamento"))))
    .withColumn("tipo_eve", F.upper(F.trim(F.col("tipo_eve"))))
    .withColumn("dia_sem_ocu", F.upper(F.trim(F.col("dia_sem_ocu"))))
    .withColumn("hora_int", F.regexp_extract(F.col("hora_ocu"), r"\d+", 0).cast("int"))
)

vehiculos = (vehiculos
    .withColumn("ano_ocu", F.col("ano_ocu").cast("int"))
    .withColumn("departamento", F.upper(F.trim(F.col("departamento"))))
    .withColumn("tipo_eve", F.upper(F.trim(F.col("tipo_eve"))))
    .withColumn("color_veh", F.upper(F.trim(F.col("color_veh"))))
)

In [0]:
FALLE_GLOB = "/Volumes/workspace/default/lab8/csv/fallecidos-lesionados/*.csv"
fallecidos_lesionados = spark.read.csv(FALLE_GLOB, header=True, inferSchema=True)

print("Listo: 'fallecidos_lesionados' cargado.")
display(fallecidos_lesionados.limit(5))

VEHICULOS_GLOB = "/Volumes/workspace/default/lab8/csv/vehiculos/*.csv"

vehiculos_raw = (spark.read
    .option("header", True)
    .option("inferSchema", False)
    .csv(VEHICULOS_GLOB)
)

vehiculos = (vehiculos_raw
    .withColumn(
        "ano_ocu_int",
        F.expr("""
            try_cast(
              nullif(regexp_replace(coalesce(ano_ocu,''), '[^0-9]', ''), '')
              as int
            )
        """)
    )
    .withColumn(
        "modelo_veh_digits",
        F.regexp_extract(F.coalesce(F.col("modelo_veh"), F.lit("")), r"(\d{4})", 1)
    )
    .withColumn(
        "modelo_veh_int",
        F.when(
            F.col("modelo_veh_digits") != "",
            F.expr("try_cast(modelo_veh_digits as int)")
        )
        .otherwise(F.lit(None).cast("int"))
    )
    .withColumn(
        "modelo_veh_int",
        F.when(F.col("modelo_veh_int").between(1900, 2099), F.col("modelo_veh_int"))
         .otherwise(F.lit(None).cast("int"))
    )
    .drop("modelo_veh_digits")
)

print("Listo: 'vehiculos' cargado.")
display(vehiculos.limit(5))


#### Análisis exploratorio

In [0]:
# ================== 1.1) Conteo y muestreo ==================
n_hechos     = hechos.count()
n_vehiculos  = vehiculos.count()
n_fall_les   = fallecidos_lesionados.count()

print(f"hechos: {n_hechos:,}")
print(f"vehiculos: {n_vehiculos:,}")
print(f"fallecidos_lesionados: {n_fall_les:,}")

print("\n[hechos]");            hechos.select("ano_ocu","mes_ocu","dia_sem_ocu","departamento","tipo_eve","hora_ocu").show(10, truncate=False)
print("\n[vehiculos]");         vehiculos.select("ano_ocu","mes_ocu","sexo_per","tipo_veh","color_veh","modelo_veh","tipo_eve").show(10, truncate=False)
print("\n[fallecidos_lesionados]"); fallecidos_lesionados.select("ano_ocu","mes_ocu","depto_ocu","zona_ocu","edad_per","tipo_eve","fall_les").show(10, truncate=False)


In [0]:
# ================== 1.2) Decribe y summary hechos ==================
hechos_num = hechos.select(
    F.expr("try_cast(ano_ocu as int)").alias("ano_ocu_int"),
    F.expr("try_cast(regexp_extract(hora_ocu, '\\\\d+', 0) as int)").alias("hora_ocu_int")
)

print("[hechos] describe()")
display(hechos_num.describe())

print("[hechos] summary()")
display(hechos_num.summary())


In [0]:
# ================== 1.3) Decribe y summary vehículos ==================
vehiculos_num = vehiculos.select("ano_ocu_int","modelo_veh_int")

print("[vehiculos] describe()")
display(vehiculos_num.describe())

print("[vehiculos] summary()")
display(vehiculos_num.summary())


In [0]:
# ================== 1.4) Decribe y summary fallecidos/lesionados ==================
falle_ll_num = fallecidos_lesionados.select(
    F.expr("try_cast(ano_ocu as int)").alias("ano_ocu_int"),
    F.expr("""
        case when lower(coalesce(edad_per,'')) in ('ignorado','ignorada','')
             then null
             else try_cast(edad_per as int)
        end
    """).alias("edad_per_int"),
    F.expr("""
        case when lower(coalesce(zona_ocu,'')) in ('ignorado','ignorada','')
             then null
             else try_cast(zona_ocu as int)
        end
    """).alias("zona_ocu_int")
)

print("[fallecidos_lesionados] describe()")
display(falle_ll_num.describe())

print("[fallecidos_lesionados] summary()")
display(falle_ll_num.summary())



In [0]:
# ================== 2) Comparación de años ==================

# Hechos: años
hechos_years = (hechos
    .select(F.expr("try_cast(ano_ocu as int)").alias("anio"))
    .where(F.col("anio").isNotNull())
    .distinct().orderBy("anio")
)
print("[hechos] años disponibles:")
hechos_years.show(200, truncate=False)

# Vehículos: si ya tienes 'ano_ocu_int', úsalo; si no, intenta try_cast
vehiculos_years_col = "ano_ocu_int" if "ano_ocu_int" in vehiculos.columns else "anio"
vehiculos_years = (vehiculos
    .select(F.col(vehiculos_years_col).alias("anio") if "ano_ocu_int" in vehiculos.columns
            else F.expr("try_cast(ano_ocu as int)").alias("anio"))
    .where(F.col("anio").isNotNull())
    .distinct().orderBy("anio")
)
print("[vehiculos] años disponibles:")
vehiculos_years.show(200, truncate=False)

# Fallecidos/Lesionados: años
falle_years = (fallecidos_lesionados
    .select(F.expr("try_cast(ano_ocu as int)").alias("anio"))
    .where(F.col("anio").isNotNull())
    .distinct().orderBy("anio")
)
print("[fallecidos_lesionados] años disponibles:")
falle_years.show(200, truncate=False)

# Comparar aparición
all_years = (hechos_years.select("anio")
             .union(vehiculos_years.select("anio"))
             .union(falle_years.select("anio"))
             .distinct())

tabla = (all_years
    .join(hechos_years.withColumn("in_hechos", F.lit(True)), ["anio"], "left")
    .join(vehiculos_years.withColumn("in_vehiculos", F.lit(True)), ["anio"], "left")
    .join(falle_years.withColumn("in_fallecidos_lesionados", F.lit(True)), ["anio"], "left")
    .select(
        "anio",
        F.coalesce("in_hechos", F.lit(False)).alias("hechos"),
        F.coalesce("in_vehiculos", F.lit(False)).alias("vehiculos"),
        F.coalesce("in_fallecidos_lesionados", F.lit(False)).alias("fallecidos_lesionados")
    )
    .orderBy("anio")
)

print("Tabla de presencia por año:")
tabla.show(200, truncate=False)


In [0]:
# ================== 3) Diferentes tipos de accidentes ==================

# Hechos
print("[hechos] tipos de accidentes:")
hechos.select(F.upper(F.trim(F.col("tipo_eve"))).alias("tipo_accidente")) \
     .distinct() \
     .orderBy("tipo_accidente") \
     .show(100, truncate=False)

# Vehículos
print("[vehículos] tipos de accidentes:")
vehiculos.select(F.upper(F.trim(F.col("tipo_eve"))).alias("tipo_accidente")) \
        .distinct() \
        .orderBy("tipo_accidente") \
        .show(100, truncate=False)


# Fallecidos/lesionados
print("[fallecidos/lesionados] tipos de accidentes:")
fallecidos_lesionados.select(F.upper(F.trim(F.col("tipo_eve"))).alias("tipo_accidente")) \
                     .distinct() \
                     .orderBy("tipo_accidente") \
                     .show(100, truncate=False)


In [0]:
# ================== 4) Departamentos únicos ==================

# Hechos
hechos.select("departamento").distinct().orderBy("departamento").show(50, truncate=False)

# Vehículos
vehiculos.select("depto_ocu").distinct().orderBy("depto_ocu").show(50, truncate=False)


# Fallecidos/lesionados
fallecidos_lesionados.select("depto_ocu").distinct().orderBy("depto_ocu").show(50, truncate=False)


#### Preguntas de análisis

In [0]:
# ================== 5) Total de accidentes por año y departamento ==================
acc_anio_depto = (hechos
    .groupBy("ano_ocu","departamento")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .orderBy("ano_ocu","departamento")
)
display(acc_anio_depto)

Databricks visualization. Run in Databricks to view.

Guatemala domina de forma consistente el conteo anual y muestra un crecimiento marcado hasta 2023, seguida por Escuintla y un grupo intermedio estable (Quetzaltenango, Santa Rosa, Sacatepéquez), mientras varios departamentos se mantienen bajos y relativamente planos. La serie revela una tendencia al alza 2013→2023 con una caída atípica en 2021, probablemente por subregistro o disrupciones operativas ligadas a la pandemia/cambios de captura.

In [0]:
# ================== 6) Día de la semana con más accidentes en 2024 ==================
acc_2023_dia = (hechos
    .filter(F.col("ano_ocu")==2023)
    .groupBy("dia_sem_ocu")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .orderBy(F.desc("accidentes"))
)
display(acc_2023_dia)

Databricks visualization. Run in Databricks to view.

En 2023 los fines de semana (domingo y sábado) concentran la mayor parte de los accidentes, lo que refleja un patrón de riesgo asociado a mayor movilidad. En contraste, los días laborales presentan cifras más bajas y relativamente estables, con miércoles como el de menor incidencia.

In [0]:
# ================== 7) Distribución por hora en "Guatemala" ==================
from pyspark.sql import functions as F

# Limpiar y convertir la hora de forma segura
base_gua = (hechos
    .filter(F.col("departamento")=="GUATEMALA")
    .withColumn(
        "hora",
        F.expr("try_cast(regexp_extract(hora_ocu, '\\\\d+', 0) as int)")
    )
    .filter((F.col("hora").isNotNull()) & (F.col("hora")>=0) & (F.col("hora")<=23))
    .select("hora")
)

# Mostrar directamente: en Plot
display(base_gua)

acc_por_hora = (
    base_gua.groupBy("hora")
    .agg(F.count("*").alias("accidentes"))
    .orderBy("hora")
)
total = acc_por_hora.agg(F.sum("accidentes")).first()[0]
acc_por_hora = acc_por_hora.withColumn("porcentaje", F.round(F.col("accidentes")/F.lit(total)*100, 2))

display(acc_por_hora)

# Top 3 y bottom 3 para comentar en conclusiones
top3 = acc_por_hora.orderBy(F.desc("accidentes")).limit(3).collect()
bot3 = acc_por_hora.orderBy(F.asc("accidentes")).limit(3).collect()

def fmt(rows):
    return ", ".join([f"{r['hora']:02d}h ({r['accidentes']})" for r in rows])

print("Picos (Top 3):", fmt(top3))
print("Valles (Bottom 3):", fmt(bot3))


Databricks visualization. Run in Databricks to view.

In [0]:
# ================== 8) Join Hechos ⨝ Vehículos (año, mes, depto, tipo) — FIX cast año ==================
from pyspark.sql import functions as F

HECHOS_GLOB    = "/Volumes/workspace/default/lab8/csv/hechos/*.csv"
VEHICULOS_GLOB = "/Volumes/workspace/default/lab8/csv/vehiculos/*.csv"

# Releer TODO como string
h_src = spark.read.csv(HECHOS_GLOB, header=True, inferSchema=False)
v_src = spark.read.csv(VEHICULOS_GLOB, header=True, inferSchema=False)

# Unificar nombres mínimos
def unify_cols(df):
    ren = {}
    if "departamento" not in df.columns and "depto_ocu" in df.columns:
        ren["depto_ocu"] = "departamento"
    if "anio" not in df.columns:
        if "ano_ocu" in df.columns: ren["ano_ocu"] = "anio_raw"
        elif "year"   in df.columns: ren["year"]   = "anio_raw"
    if "mes" not in df.columns:
        if "mes_ocu" in df.columns: ren["mes_ocu"] = "mes_raw"
        elif "month"  in df.columns: ren["month"]  = "mes_raw"
    if "tipo_accidente" not in df.columns:
        if "tipo_eve" in df.columns: ren["tipo_eve"] = "tipo_raw"
        elif "tipo"    in df.columns: ren["tipo"]    = "tipo_raw"
    out = df
    for k,v in ren.items():
        out = out.withColumnRenamed(k, v)
    return out

h = unify_cols(h_src)
v = unify_cols(v_src)

# ---------- Helpers (aceptan str o Column) ----------
def _as_col(x):
    return F.col(x) if isinstance(x, str) else x

def clean_depto(x):
    c = _as_col(x)
    c = F.upper(F.trim(c))
    c = F.translate(c, "ÁÉÍÓÚáéíóúÑñ", "AEIOUaeiouNn")
    c = F.regexp_replace(c, r"\?", "E")
    c = F.regexp_replace(c, r"\s+", " ")
    return c

MESES = {
    "ENERO":"01","FEBRERO":"02","MARZO":"03","ABRIL":"04","MAYO":"05","JUNIO":"06",
    "JULIO":"07","AGOSTO":"08","SEPTIEMBRE":"09","SETIEMBRE":"09","OCTUBRE":"10",
    "NOVIEMBRE":"11","DICIEMBRE":"12"
}
def clean_mes(x):
    m = _as_col(x)
    m = F.upper(F.trim(m))
    m = F.translate(m, "ÁÉÍÓÚáéíóú", "AEIOUaeiou")
    expr = None
    for k,v in MESES.items():
        expr = F.when(m == F.lit(k), F.lit(v)) if expr is None else expr.when(m == F.lit(k), F.lit(v))
    num = F.lpad(F.regexp_extract(m, r"\d+", 0), 2, "0")
    expr = expr.otherwise(num) if expr is not None else num
    return F.when(expr == "", None).otherwise(expr)

def clean_anio(x):
    c = _as_col(x)
    extr = F.regexp_extract(c.cast("string"), r"\d{4}", 0)        # "" si no hay año
    return F.when(F.length(extr) > 0, extr.cast("int")).otherwise(F.lit(None).cast("int"))

def clean_tipo(x):
    c = _as_col(x)
    return F.upper(F.trim(c))

def pick(df, *names):
    for n in names:
        if n in df.columns:
            return F.col(n)
    return F.lit(None)

# Canónicas
h = (h
     .withColumn("departamento",   clean_depto("departamento"))
     .withColumn("anio",           clean_anio(pick(h, "anio_raw", "anio")))
     .withColumn("mes",            clean_mes(pick(h, "mes_raw",  "mes")))
     .withColumn("tipo_accidente", clean_tipo(pick(h, "tipo_raw","tipo_accidente")))
     .select("anio","mes","departamento","tipo_accidente")
)
v = (v
     .withColumn("departamento",   clean_depto("departamento"))
     .withColumn("anio",           clean_anio(pick(v, "anio_raw", "anio")))
     .withColumn("mes",            clean_mes(pick(v, "mes_raw",  "mes")))
     .withColumn("tipo_accidente", clean_tipo(pick(v, "tipo_raw","tipo_accidente")))
     .select("anio","mes","departamento","tipo_accidente")
)

# Filtrar llaves válidas
VALID_MES = r"^(0[1-9]|1[0-2])$"
h_k = h.filter( (F.col("anio").isNotNull()) & (F.col("mes").rlike(VALID_MES)) & (F.col("departamento").isNotNull()) & (F.col("tipo_accidente").isNotNull()) )
v_k = v.filter( (F.col("anio").isNotNull()) & (F.col("mes").rlike(VALID_MES)) & (F.col("departamento").isNotNull()) & (F.col("tipo_accidente").isNotNull()) )

# Agregar y unir
H  = h_k.groupBy("anio","mes","departamento","tipo_accidente").agg(F.count(F.lit(1)).alias("accidentes"))
V  = v_k.groupBy("anio","mes","departamento","tipo_accidente").agg(F.count(F.lit(1)).alias("vehiculos"))
HV = H.join(V, on=["anio","mes","departamento","tipo_accidente"], how="inner")

# Resultado
hv_count = HV.count()
print(f"Registros combinados (H⨝V): {hv_count:,}")

# Vista + diagnósticos
display(HV.orderBy("anio","mes","departamento","tipo_accidente").limit(20))
print("Claves en Hechos   :", H.count())
print("Claves en Vehículos:", V.count())
print("Claves en el Join  :", HV.count())

no_en_V = H.join(V, on=["anio","mes","departamento","tipo_accidente"], how="left_anti")
no_en_H = V.join(H, on=["anio","mes","departamento","tipo_accidente"], how="left_anti")
print("Solo en Hechos (ejemplos):");    display(no_en_V.limit(10))
print("Solo en Vehículos (ejemplos):"); display(no_en_H.limit(10))


Con la llave compuesta (mes, año, departamento y tipo de accidente), conseguimos 8,694 registros combinados, lo que representa empatar aproximadamente el 96% de las 9,060 claves de vehículos y cerca del 86% de las 10,056 claves de hechos; en otras palabras, aunque el cruce es alto, no es absoluto. Los anti-joins revelan las razones de la falta de pares: diferencias en la codificación y las tildes en tipo_accidente, errores tipográficos en departamento y valores poco claros como IGNORADO y algunos años sin dígitos válidos.

In [0]:
# ================== 9) Promedio de vehículos por accidente por departamento ==================
from pyspark.sql import functions as F

# Partimos del join del inciso 8
assert 'HV' in globals(), "No encuentro 'HV'. Ejecuta primero el inciso 8."
hvdf = HV

# Agregar por departamento: promedio = sum(vehiculos) / sum(accidentes)
prom_depto = (
    hvdf.groupBy("departamento")
        .agg(
            F.sum("vehiculos").alias("vehiculos_tot"),
            F.sum("accidentes").alias("accidentes_tot")
        )
        .withColumn(
            "vehiculos_por_accidente",
            F.when(F.col("accidentes_tot") > 0, F.col("vehiculos_tot")/F.col("accidentes_tot"))
             .otherwise(F.lit(None))
        )
        .select("departamento", "vehiculos_por_accidente", "vehiculos_tot", "accidentes_tot")
        .orderBy(F.desc("vehiculos_por_accidente"))
)

display(prom_depto)  # vista previa

# Guardar como Parquete
OUT_DIR = "/Volumes/workspace/default/lab8/out"
OUT_PATH = f"{OUT_DIR}/prom_vehiculos_accidente"
(
    prom_depto
    .coalesce(1)              
    .write.mode("overwrite")
    .parquet(OUT_PATH)
)
print(f"Guardado Parquet en: {OUT_PATH}")

# Recargar desde Parquet
prom_reload = spark.read.parquet(OUT_PATH)

# Top 10 departamentos con más vehículos/accidente
top10 = prom_reload.orderBy(F.desc("vehiculos_por_accidente")).limit(10)
display(top10)


Databricks visualization. Run in Databricks to view.

In [0]:
# ================== 10) Top 5 colores de vehiculos con más accidentes ==================
(vehiculos
 .filter(F.col("color_veh").isNotNull() & (F.length(F.trim(F.col("color_veh"))) > 0))
 .filter(~F.upper(F.col("color_veh")).isin("IGNORADO"))
 .groupBy("color_veh")
 .agg(F.count(F.lit(1)).alias("accidentes"))
 .orderBy(F.desc("accidentes"))
 .limit(5)
 .show(truncate=False)
)


In [0]:
# ================== 11) Lesionados por atropello 2024 - por mes ==================

falle_2024 = (fallecidos_lesionados
    .withColumn("ano_int", F.expr("try_cast(ano_ocu as int)"))
    .withColumn("tipo_eve_lc", F.lower(F.col("tipo_eve")))
    .withColumn("fall_les_lc", F.lower(F.col("fall_les")))
    .withColumn("mes_lc", F.lower(F.col("mes_ocu")))
    .filter( (F.col("ano_int")==2023) &
             (F.col("tipo_eve_lc")=="atropello") &
             (F.col("fall_les_lc")=="lesionado") )
)

month_order = {
    "enero":1, "febrero":2, "marzo":3, "abril":4, "mayo":5, "junio":6,
    "julio":7, "agosto":8, "septiembre":9, "octubre":10, "noviembre":11, "diciembre":12
}
month_map = F.create_map([x for kv in month_order.items() for x in (F.lit(kv[0]), F.lit(kv[1]))])

lesiones_mensual = (falle_2024
    .groupBy("mes_lc")
    .agg(F.count(F.lit(1)).alias("lesionados"))
    .withColumn("mes_num", F.element_at(month_map, F.col("mes_lc")))
    .orderBy("mes_num")
    .select(F.col("mes_lc").alias("mes"), "lesionados")
)

display(lesiones_mensual)

# Gráfica
import matplotlib.pyplot as plt

pdf = lesiones_mensual.toPandas()
pdf = pdf.sort_values(by="mes", key=lambda s: s.map(month_order))

plt.figure(figsize=(8,4))
plt.plot(pdf["mes"], pdf["lesionados"], marker="o")
plt.title("Lesionados por atropello en 2024 (por mes)")
plt.xlabel("Mes")
plt.ylabel("Cantidad de lesionados")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()



In [0]:
# ================== 12) Fallecidos por tipo de accidente ==================

# Unificar por llave
fl_src = spark.read.csv(FALLE_GLOB, header=True, inferSchema=False)

def unify_cols_fl(df):
    ren = {}
    if "departamento" not in df.columns and "depto_ocu" in df.columns:
        ren["depto_ocu"] = "departamento"
    if "anio" not in df.columns:
        if "ano_ocu" in df.columns: ren["ano_ocu"] = "anio_raw"
    if "mes" not in df.columns:
        if "mes_ocu" in df.columns: ren["mes_ocu"] = "mes_raw"
    if "tipo_accidente" not in df.columns:
        if "tipo_eve" in df.columns: ren["tipo_eve"] = "tipo_raw"
    out = df
    for k,v in ren.items():
        out = out.withColumnRenamed(k, v)
    return out

fl = unify_cols_fl(fl_src)

fl = (fl
      .withColumn("departamento",   clean_depto("departamento"))
      .withColumn("anio",           clean_anio(pick(fl, "anio_raw", "anio")))
      .withColumn("mes",            clean_mes(pick(fl, "mes_raw",  "mes")))
      .withColumn("tipo_accidente", clean_tipo(pick(fl, "tipo_raw","tipo_accidente")))
      .withColumn("fall_les_norm",  F.upper(F.trim(F.col("fall_les"))))
      .select("anio","mes","departamento","tipo_accidente","fall_les_norm")
)

VALID_MES = r"^(0[1-9]|1[0-2])$"
fl_k = fl.filter(
    (F.col("anio").isNotNull()) &
    (F.col("mes").rlike(VALID_MES)) &
    (F.col("departamento").isNotNull()) &
    (F.col("tipo_accidente").isNotNull())
)

FL = (fl_k
      .groupBy("anio","mes","departamento","tipo_accidente")
      .agg(
          F.sum(F.when(F.col("fall_les_norm")=="FALLECIDO", 1).otherwise(0)).alias("fallecidos"),
          F.sum(F.when(F.col("fall_les_norm")=="LESIONADO", 1).otherwise(0)).alias("lesionados")
      )
      .withColumn("total_fl", F.col("fallecidos")+F.col("lesionados"))
)

HV_FL = (HV.join(FL, on=["anio","mes","departamento","tipo_accidente"], how="left")
           .fillna({"fallecidos":0, "lesionados":0, "total_fl":0})
)

print(f"Registros combinados (H⨝V⨝FL): {HV_FL.count():,}")
display(HV_FL.orderBy("anio","mes","departamento","tipo_accidente").limit(20))

In [0]:
# Fallecidos por accidente
fallecidos_por_tipo = (
    HV_FL.groupBy("tipo_accidente")
         .agg(F.sum(F.col("fallecidos")).alias("total_fallecidos"))
         .orderBy(F.desc("total_fallecidos"))
)

display(fallecidos_por_tipo)

# Gráfica
import matplotlib.pyplot as plt

pdf = fallecidos_por_tipo.toPandas()

plt.figure(figsize=(8, 5))
plt.barh(pdf["tipo_accidente"], pdf["total_fallecidos"])
plt.xlabel("Total de fallecidos")
plt.ylabel("Tipo de accidente")
plt.title("Fallecidos por tipo de accidente")
plt.gca().invert_yaxis()  # para que el mayor quede arriba
plt.tight_layout()


In [0]:
# ================== 13) Clasificación de accidentes por franja horaria ==================
from pyspark.sql import functions as F

# Limpiamos y convertimos la hora a entero si no lo está
hechos_hora = (
    hechos
    .withColumn("hora_int", F.when(F.col("hora_ocu").rlike("^[0-9]+$"), F.col("hora_ocu").cast("int")).otherwise(None))
    .filter(F.col("hora_int").isNotNull())
)

# Clasificamos las franjas horarias
hechos_franjas = (
    hechos_hora
    .withColumn(
        "franja_horaria",
        F.when((F.col("hora_int") >= 6) & (F.col("hora_int") < 12), "Mañana")
         .when((F.col("hora_int") >= 12) & (F.col("hora_int") < 18), "Tarde")
         .when((F.col("hora_int") >= 18) & (F.col("hora_int") < 24), "Noche")
         .when((F.col("hora_int") >= 0)  & (F.col("hora_int") < 6),  "Madrugada")
         .otherwise("Desconocido")
    )
)

# Agrupamos para contar accidentes por franja
acc_por_franja = (
    hechos_franjas
    .groupBy("franja_horaria")
    .agg(F.count("*").alias("accidentes"))
    .orderBy(F.desc("accidentes"))
)

display(acc_por_franja)


In [0]:
# ================== 14) Ratio fallecidos/accidente por departamento ==================
from pyspark.sql import functions as F

FALLECIDOS_GLOB = "/Volumes/workspace/default/lab8/csv/fallecidos-lesionados/*.csv"
OUT_PATH        = "/Volumes/workspace/default/lab8/resultados/ratio_fallecidos_por_accidente"

# Helpers de limpieza
def clean_depto(col):
    c = F.upper(F.trim(F.col(col)))
    c = F.translate(c, "ÁÉÍÓÚáéíóúÑñ", "AEIOUaeiouNn")
    c = F.regexp_replace(c, r"\?", "E")
    c = F.regexp_replace(c, r"\s+", " ")
    return c

# Base de accidentes (hechos) por departamento
hec = hechos.withColumnRenamed("depto_ocu","departamento") if "depto_ocu" in hechos.columns else hechos
hec = hec.withColumn("departamento", clean_depto("departamento"))

acc_dep = (hec
    .groupBy("departamento")
    .agg(F.count(F.lit(1)).alias("total_accidentes"))
)

# Víctimas: detectar fallecidos desde `fall_les`
vic = spark.read.csv(FALLECIDOS_GLOB, header=True, inferSchema=False)
vic = vic.withColumnRenamed("depto_ocu","departamento") if "depto_ocu" in vic.columns else vic
vic = vic.withColumn("departamento", clean_depto("departamento"))

# Normaliza texto de fall_les
fall = F.upper(F.trim(F.col("fall_les"))) if "fall_les" in vic.columns else F.lit(None)

fallecido_flag = F.when(
    fall.rlike(r"^FALLEC") | (fall == F.lit("F")) |
    (F.when(F.col("fall_les").rlike(r"^\d+$"), F.col("fall_les").cast("int")).otherwise(F.lit(0)) > 0),
    F.lit(1)
).otherwise(F.lit(0))

vic_dep = (vic
    .withColumn("fallecido", fallecido_flag)
    .groupBy("departamento")
    .agg(F.sum("fallecido").alias("total_fallecidos"))
)

# Ratio por departamento
ratio_df = (acc_dep.join(vic_dep, on="departamento", how="left")
    .fillna({"total_fallecidos": 0})
    .withColumn("ratio_fallecidos",
                F.round(F.col("total_fallecidos") / F.col("total_accidentes"), 4))
    .orderBy(F.desc("ratio_fallecidos"))
)

display(ratio_df)

# Guardar en Parquet y recargar top-10 para graficar con display
ratio_df.write.mode("overwrite").parquet(OUT_PATH)
print(f"Guardado en: {OUT_PATH}")

top10 = spark.read.parquet(OUT_PATH).orderBy(F.desc("ratio_fallecidos")).limit(10)
display(top10)


In [0]:
# ================== 15) Barras comparativas FALLECIDO vs LESIONADO por grupo de edad ==================
from pyspark.sql import functions as F

VIC_GLOB = "/Volumes/workspace/default/lab8/csv/fallecidos-lesionados/*.csv"
vic = spark.read.csv(VIC_GLOB, header=True, inferSchema=False)

cands = [c for c in vic.columns if "edad" in c.lower()]
edad_col = None
for c in cands:
    if vic.filter(F.col(c).cast("string").rlike(r"\d+")).limit(1).count() > 0:
        edad_col = c
        break
if edad_col is None:
    raise ValueError("No se encontró una columna de edad utilizable en el CSV.")

print("Columna elegida para edad:", edad_col)

fall = F.upper(F.trim(F.col("fall_les").cast("string")))
flag = F.when(F.col("fall_les").rlike(r"^\d+$"), F.col("fall_les").cast("int"))
estado = (
    F.when((fall.rlike(r"^FALLEC") | (fall=="F") | (flag==1)),  "FALLECIDO")
     .when((fall.rlike(r"^LESION") | (fall=="L") | (flag==0)), "LESIONADO")
     .otherwise("DESCONOCIDO")
)

digits   = F.regexp_extract(F.col(edad_col).cast("string"), r"\d+", 0)
edad_int = F.when(digits=="", None).otherwise(digits.cast("int"))

grupo = (
    F.when((edad_int>=0)  & (edad_int<=11), "00–11")
     .when((edad_int>=12) & (edad_int<=17), "12–17")
     .when((edad_int>=18) & (edad_int<=29), "18–29")
     .when((edad_int>=30) & (edad_int<=44), "30–44")
     .when((edad_int>=45) & (edad_int<=59), "45–59")
     .when((edad_int>=60),                  "60+")
     .otherwise("SIN_EDAD")
)

vic_bins = (vic
    .withColumn("estado", estado)
    .withColumn("edad",   edad_int)
    .withColumn("grupo_edad", grupo)
)

orden = (F.when(F.col("grupo_edad")=="00–11",0)
          .when(F.col("grupo_edad")=="12–17",1)
          .when(F.col("grupo_edad")=="18–29",2)
          .when(F.col("grupo_edad")=="30–44",3)
          .when(F.col("grupo_edad")=="45–59",4)
          .when(F.col("grupo_edad")=="60+",  5)
          .otherwise(6))

res = (vic_bins
    .filter(F.col("estado").isin("FALLECIDO","LESIONADO"))
    .groupBy("grupo_edad","estado")
    .agg(F.count(F.lit(1)).alias("personas"))
    .withColumn("orden", orden)
    .filter(F.col("grupo_edad") != "SIN_EDAD")
    .orderBy("orden","estado")
)

display(res)


In [0]:
import matplotlib.pyplot as plt

pdf = res.toPandas()

pivot = pdf.pivot(index='grupo_edad', columns='estado', values='personas').fillna(0)

# Ordenar correctamente los grupos de edad
orden = ["00–11", "12–17", "18–29", "30–44", "45–59", "60+"]
pivot = pivot.reindex(orden)

# Crear gráfico de barras comparativas
ax = pivot.plot(kind='bar', figsize=(8,5), width=0.8)

plt.title("Comparativa de fallecidos y lesionados por grupo de edad")
plt.xlabel("Grupo de edad")
plt.ylabel("Número de personas")
plt.xticks(rotation=0)
plt.legend(title="Estado")
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


In [0]:
# ================== 16) Accidentes y fallecidos por zona (Municipio GUATEMALA) ==================
from pyspark.sql import functions as F

HECHOS_GLOB = "/Volumes/workspace/default/lab8/csv/hechos/*.csv"
VIC_GLOB    = "/Volumes/workspace/default/lab8/csv/fallecidos-lesionados/*.csv"

hec_src = spark.read.csv(HECHOS_GLOB, header=True, inferSchema=False)
vic_src = spark.read.csv(VIC_GLOB,   header=True, inferSchema=False)

def alias(df):
    ren = {}
    if "depto_ocu" in df.columns and "departamento" not in df.columns: ren["depto_ocu"] = "departamento"
    if "ano_ocu"   in df.columns and "anio"         not in df.columns: ren["ano_ocu"]   = "anio"
    if "mes_ocu"   in df.columns and "mes"          not in df.columns: ren["mes_ocu"]   = "mes"
    if "tipo_eve"  in df.columns and "tipo_accidente" not in df.columns: ren["tipo_eve"] = "tipo_accidente"
    if "zona_ocu"  in df.columns and "zona"         not in df.columns: ren["zona_ocu"]  = "zona"
    out = df
    for k,v in ren.items():
        out = out.withColumnRenamed(k, v)
    return out

hec = alias(hec_src)
vic = alias(vic_src)

def clean_depto(c):
    return F.regexp_replace(
        F.translate(F.upper(F.trim(F.col(c))), "ÁÉÍÓÚáéíóúÑñ", "AEIOUaeiouNn"),
        r"\?", "E"
    )

MESES = {"ENERO":"01","FEBRERO":"02","MARZO":"03","ABRIL":"04","MAYO":"05","JUNIO":"06",
         "JULIO":"07","AGOSTO":"08","SEPTIEMBRE":"09","SETIEMBRE":"09","OCTUBRE":"10",
         "NOVIEMBRE":"11","DICIEMBRE":"12"}

def clean_mes(c):
    m = F.translate(F.upper(F.trim(F.col(c))), "ÁÉÍÓÚáéíóú", "AEIOUaeiou")
    expr = None
    for k,v in MESES.items():
        expr = F.when(m==k, v) if expr is None else expr.when(m==k, v)
    num = F.lpad(F.regexp_extract(m, r"\d+", 0), 2, "0")
    return (expr.otherwise(num) if expr is not None else num)

def clean_anio(c):  return F.regexp_extract(F.col(c).cast("string"), r"\d{4}", 0).cast("int")
def clean_tipo(c):  return F.upper(F.trim(F.col(c)))

hec = (hec
    .withColumn("departamento", clean_depto("departamento"))
    .withColumn("mes",          clean_mes("mes"))
    .withColumn("anio",         clean_anio("anio"))
    .withColumn("tipo_accidente", clean_tipo("tipo_accidente"))
)

vic = (vic
    .withColumn("departamento", clean_depto("departamento"))
    .withColumn("mes",          clean_mes("mes"))
    .withColumn("anio",         clean_anio("anio"))
    .withColumn("tipo_accidente", clean_tipo("tipo_accidente"))
    .withColumn("zona_num", F.regexp_extract(F.upper(F.trim(F.col("zona"))), r"\d+", 0))
)

VALID_MES = r"^(0[1-9]|1[0-2])$"
hec_k = hec.filter( F.col("anio").isNotNull() & F.col("mes").rlike(VALID_MES)
                    & F.col("departamento").isNotNull() & F.col("tipo_accidente").isNotNull() )

vic_k = vic.filter( F.col("anio").isNotNull() & F.col("mes").rlike(VALID_MES)
                    & F.col("departamento").isNotNull() & F.col("tipo_accidente").isNotNull()
                    & (F.col("zona_num")!="") )

# Accidentes por llave (en GUATEMALA)
H = (hec_k.filter(F.col("departamento")=="GUATEMALA")
        .groupBy("anio","mes","departamento","tipo_accidente")
        .agg(F.count(F.lit(1)).alias("accidentes"))
)

# Llave + zona desde víctimas (en GUATEMALA)
KZ = (vic_k.filter(F.col("departamento")=="GUATEMALA")
         .select("anio","mes","departamento","tipo_accidente","zona_num")
         .distinct()
)

# Accidentes por zona (usando la zona traída de víctimas)
acc_zona = (H.join(KZ, on=["anio","mes","departamento","tipo_accidente"], how="inner")
              .groupBy("zona_num")
              .agg(F.sum("accidentes").alias("accidentes")))

# Fallecidos por zona
fall = F.upper(F.trim(F.col("fall_les")))
flag = F.when(F.col("fall_les").rlike(r"^\d+$"), F.col("fall_les").cast("int"))
es_fallecido = (fall.rlike(r"^FALLEC") | (fall=="F") | (flag==1))

fall_zona = (vic_k.filter( (F.col("departamento")=="GUATEMALA") & es_fallecido )
                .groupBy("zona_num")
                .agg(F.count(F.lit(1)).alias("fallecidos")))

# Unir indicadores y ordenar por zona numérica
zona_ind = (acc_zona.join(fall_zona, on="zona_num", how="outer")
                     .na.fill(0)
                     .withColumnRenamed("zona_num","zona")
                     .orderBy(F.col("zona").cast("int")))

display(zona_ind)


In [0]:
import matplotlib.pyplot as plt

pdf = zona_ind.toPandas()
pdf["zona"] = pdf["zona"].astype(int)
pdf = pdf.sort_values("zona")

ax = (
    pdf.set_index("zona")[["accidentes", "fallecidos"]]
       .plot(kind="bar", figsize=(11,5), width=0.85)
)

plt.title("Municipio de Guatemala: Accidentes vs Fallecidos por zona")
plt.xlabel("Zona")
plt.ylabel("Cantidad")
plt.xticks(rotation=0)
plt.legend(title="Indicador")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


In [0]:
# ================== 17) Porcentaje de accidentes por sexo del conductor ==================
from pyspark.sql import functions as F

VEH_GLOB = "/Volumes/workspace/default/lab8/csv/vehiculos/*.csv"

veh = spark.read.csv(VEH_GLOB, header=True, inferSchema=False)

# Detectar columna de sexo
cands = [c for c in veh.columns if "sexo" in c.lower()]
sexo_col = None
for c in cands:
    if veh.filter(F.col(c).isNotNull()).limit(1).count() > 0:
        sexo_col = c
        break

if sexo_col is None:
    raise ValueError("No se encontró una columna de sexo utilizable en los CSV de vehículos.")

print("Columna elegida para sexo:", sexo_col)

# Normalizar los valores
sexo_norm = (
    F.when(F.upper(F.trim(F.col(sexo_col))).isin("M","MASCULINO","HOMBRE"), "HOMBRE")
     .when(F.upper(F.trim(F.col(sexo_col))).isin("F","FEMENINO","MUJER"), "MUJER")
     .otherwise("DESCONOCIDO")
)

veh_norm = veh.withColumn("sexo_norm", sexo_norm)

# Calcular porcentajes
total = veh_norm.count()
res = (
    veh_norm.groupBy("sexo_norm")
    .agg(F.count(F.lit(1)).alias("accidentes"))
    .withColumn("porcentaje", F.round(F.col("accidentes") * 100 / total, 2))
    .filter(F.col("sexo_norm") != "DESCONOCIDO")
)

# Guardar en Parquet
OUT_PATH = "/Volumes/workspace/default/lab8/parquet/accidentes_por_sexo"
res.write.mode("overwrite").parquet(OUT_PATH)

# Volver a cargar y mostrar
res_loaded = spark.read.parquet(OUT_PATH)
display(res_loaded)

In [0]:
import matplotlib.pyplot as plt

pdf = res_loaded.toPandas().sort_values("sexo_norm")

plt.figure(figsize=(6,6))
plt.pie(pdf["porcentaje"], labels=pdf["sexo_norm"], autopct='%1.1f%%', startangle=90)
plt.title("Porcentaje de accidentes según sexo del conductor")
plt.axis("equal")
plt.show()
